In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']



In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
polynomial_converter = PolynomialFeatures(degree = 3, include_bias=False)

In [6]:
poly_features = polynomial_converter.fit_transform(X)

In [7]:
poly_features[0, :]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [10]:
# Scale the data

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()   # feature scaling – standardisation

In [13]:
scaler.fit(X_train)     # Note, you only fit (i.e. compute mean, std) on the X_train data.

StandardScaler()

In [14]:
# Now you transform, using the same scaler, both X_train and X_test

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Now perform Ridge regression 

In [16]:
from sklearn.linear_model import Ridge

In [17]:
ridge_model = Ridge(alpha = 10)     # Perform linear regression (on polynomialised features) with 'lambda' = 10

In [18]:
ridge_model.fit(X_train_scaled, y_train)

Ridge(alpha=10)

In [19]:
ridge_model.coef_           # These are the 'theta' parameters, found by fitting on the training set with 'lambda' = 10

array([ 2.11063462,  0.62207087,  0.06822351,  0.20543482,  1.83561135,
        0.33399306,  0.02544162,  0.15472622, -0.02220834, -0.62586462,
        0.41216201, -0.49628796,  1.11873234,  0.24222164, -0.00821273,
       -0.20160416, -0.14825878, -0.06688285, -0.06004278])

In [20]:
# Use the above theta coefficients to predict on X_test data set

test_predictions = ridge_model.predict(X_test_scaled)

In [21]:
# Now want to compute errors on the test set

In [22]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
MAE = mean_absolute_error(test_predictions, y_test)
MSE = mean_squared_error(test_predictions, y_test)
RMSE = np.sqrt(MSE)

In [24]:
MAE, MSE, RMSE

(0.5774404204714181, 0.8003783071528392, 0.8946386461319672)

In [25]:
# How did it perform on the training set? 
train_predictions = ridge_model.predict(X_train_scaled)
MAE_train = mean_absolute_error(train_predictions, y_train)
MSE_train = mean_squared_error(train_predictions, y_train)
RMSE_train = np.sqrt(MSE_train)

In [26]:
(MAE_train, MSE_train, RMSE_train)

(0.5288348183025319, 0.7211075569495089, 0.8491805208255244)

In [27]:
# OK, this makes sense. Errors for training set are lower

In [28]:
# Now we want to find the optimal lambda ('alpha') value by using CV

In [29]:
# The process is similar, except we use the Ridge_CV class rather than Ridge

In [30]:
from sklearn.linear_model import RidgeCV

In [31]:
from sklearn.metrics import SCORERS

In [32]:
SCORERS.keys()  
# This gives a list of error computations we can choose from. 
# Note that in sklearn, higher is always better, so the library computes the negative of the error.

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [33]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error') # Pass in a tuple of alpha values; specify error metric

In [34]:
ridge_cv_model.fit(X_train_scaled, y_train)         

# This tries out the different alpha values and computes the average error by doing K-folds cross validation.
# Then using the optimal alpha, it finds the theta parameters for the X_train_scaled, y_train values

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [35]:
# Check the optimal alpha / lambda:
ridge_cv_model.alpha_ 

0.1

In [36]:
# Check the theta parameters
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [37]:
# Now use model to predict on test set
test_predictions = ridge_cv_model.predict(X_test_scaled)

In [38]:
# Check errors on  test dataset

MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [39]:
(MAE, MSE, RMSE)

(0.4273774884329612, 0.38201298815235735, 0.618071992693697)

In [40]:
# Check errors on training dataset

train_predictions = ridge_cv_model.predict(X_train_scaled)

MAE_train = mean_absolute_error(y_train,train_predictions)
MSE_train = mean_squared_error(y_train,train_predictions)
RMSE_train = np.sqrt(MSE_train)

In [41]:
(MAE_train, MSE_train, RMSE_train)

(0.30941321056334764, 0.2207985871984561, 0.4698921016557483)

In [42]:
# OK, makes sense. Training errors are lower.

In [43]:
# Now we move onto LASSO regression

In [44]:
from sklearn.linear_model import LassoCV
lasso_cv_model = LassoCV(eps=0.001,n_alphas=1000,cv=5, max_iter = 1000000)      # eps = alpha_min/alpha_max; cv = the 'K' value in K-folds.



In [45]:
lasso_cv_model.fit(X_train_scaled,y_train)

# like with Ridge_CV, the instance of LassoCV will compute the errors for all the different lambda values. 
# once we fit it to the training set, its attributes include the optimal lambda value and the theta parameters for that lambada value.

LassoCV(cv=5, max_iter=1000000, n_alphas=1000)

In [46]:
lasso_cv_model.alpha_

0.004943070909225827

In [47]:
lasso_cv_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

In [48]:
test_predictions = lasso_cv_model.predict(X_test_scaled)

In [49]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [50]:
MAE, MSE, RMSE

(0.43350346185900707, 0.3676167574199059, 0.6063140748984027)

In [51]:
# Compare with training set error

train_predictions = lasso_cv_model.predict(X_train_scaled)
MAE_train = mean_absolute_error(y_train, train_predictions)
MSE_train = mean_squared_error(y_train, train_predictions)
RMSE_train = np.sqrt(MSE_train)

In [52]:
MAE_train, MSE_train, RMSE_train

(0.33070562256744884, 0.2656058737019553, 0.5153696476335751)

In [53]:
# OK good, so lower, but not by a huge amount.

In [54]:
# Alright. Now we use elastic net regularisation

In [55]:
from sklearn.linear_model import ElasticNetCV

In [56]:
elastic_model= ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], max_iter = 100000)    

# The regularisation part of the error function: 
# a * ||w||_1 + 0.5 * b * ||w||_2^2
# alpha = a + b and l1_ratio = a / (a + b)

In [57]:
elastic_model.fit(X_train_scaled, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=100000)

In [58]:
elastic_model.l1_ratio_

1.0

In [59]:
# Interesting. So the model thinks optimally we should only use lasso regularisation.

In [60]:
# Make predictions on test set
test_predictions = elastic_model.predict(X_test_scaled)

In [61]:
# Compute errors on test set

MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [62]:
MAE, MSE, RMSE

(0.43350346185900707, 0.3676167574199059, 0.6063140748984027)

In [63]:
# Compute errors on training set
train_predictions = elastic_model.predict(X_train_scaled)
MAE_train = mean_absolute_error(y_train, train_predictions)
MSE_train = mean_squared_error(y_train, train_predictions)
RMSE_train = np.sqrt(MSE_train)

In [64]:
MAE_train, MSE_train, RMSE_train

(0.33070562256744884, 0.2656058737019553, 0.5153696476335751)

In [65]:
# Again, makes sense. Training errors are lower than test errors. 
# Also note, since l1_ratio = 1, the errors are the same as when we just did Lasso regularisation.